# Loading of the dataset and First preprocessing step
for this project I chose the dataset available on Hugging face ncbi-virus-complete-dna-v230722, but reducing its size, the original dataset contains over 2 million rows, but for a matter of complexity and capacity of the pc I took a portion containing the sequence that have a lenght less than 4000 

To download the complete dataset, you can run the cell below which will automatically check if the dataset is in the folder, otherwise the download

In [1]:
import json
import glob

import numpy as np
from datasets import load_dataset
import pandas as pd
import os
from imblearn.over_sampling import SMOTEN
from tqdm.notebook import tqdm


path_dataframe = '/Volumes/Seagate Bas/Vito/ML/Dataset/'
path_multiclass = path_dataframe + 'MulticlassDatasets/'
df_name = 'ncbi-virus-all-complete-nucleotides-2023-07-22'

# if we want to load the file from parquet or from jsonl
parquet = False

# number of rows selected
n = 200000
# max length of the sequence
max_len  = 4000

To reduce the dataset size we take the at most 200 000 rows of the dataset

In [2]:
if not glob.glob(path_dataframe + f'{df_name}.jsonl'):
    print('Downloading dataset...')
    # load dataset from hugging face
    df = load_dataset("LKarlo/ncbi-virus-complete-dna-v230722")['train']
    # Save the dataset on parquet extention
    df.to_parquet(path_dataframe + f'{df_name}.parquet')
    df = df.sample(n=n)
else: 
    if os.path.exists(path_dataframe + f'ncbi-virus-{n}-dna.jsonl'):
        print(f'Loading the dataset consisting of at most {n} rows in jsonl format')
        df = None
        with open(path_dataframe + f'ncbi-virus-{n}-dna.jsonl') as f:
            for index, line in tqdm(enumerate(f)):
                data = json.loads(line)
                if df is None:
                    df = pd.DataFrame(data=data, index=[0])
                else:
                    df.loc[len(df)] = data
    else:    
        print(f'Loading at most {n} raws from the entire dataset from jsonl...')
        df = None
        with open(path_dataframe + f'{df_name}.jsonl') as f:
            for index, line in tqdm(enumerate(f)):
                data = json.loads(line)
                if df is None:
                    df = pd.DataFrame(data=data, index=[0])
                else:
                    if index < n:
                        continue
                    elif len(df) == n:
                        break
                    if int(data['Length']) < max_len:
                        df.loc[len(df)] = data
df.head()                

Loading the dataset consisting of 200000 rows in jsonl format


0it [00:00, ?it/s]

,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_079174.1,2023-05-17T00:00:00Z,Erwinia phage Fifi44,Loessnervirus,Chaseviridae,dsDNA,53559,RefSeq,Erwinia amylovora,2020-09,TGTTCCATGTGAGTTACTGAGTTCCGTAATCCTATTATAAAGGCCC...
1,OQ599920.1,2023-06-20T00:00:00Z,Banfec anellovirus 1,Wawtorquevirus,Anelloviridae,ssDNA(-),2471,GenBank,Canis latrans,2021-12-17,CCCCCCCCCTTAAACCCCCCCCGCAGGGGGGCTCCGCCCCCCCTGC...
2,OQ599921.1,2023-06-20T00:00:00Z,Banfec anellovirus 2,Wawtorquevirus,Anelloviridae,ssDNA(-),2479,GenBank,Canis latrans,2021-12-17,CCCCCCCCGCACCCCCCCAAGGGGCTCCGCCCCTTAAACCCCATTG...
3,OQ599922.1,2023-06-20T00:00:00Z,Banfec circovirus 1,Circovirus,Circoviridae,ssDNA(+/-),1680,GenBank,Canis latrans,2021-04-24,TATTACCGGAGACTTTGGAACCGTGGAACAGGCCGATCTGTAATGA...
4,OQ599923.1,2023-06-20T00:00:00Z,Banfec circovirus 1,Circovirus,Circoviridae,ssDNA(+/-),1680,GenBank,Canis latrans,2021-04-23,TATTACCGGAGACTTTGGAACCGTGGAACAGGCCGATCTGTAATGA...


Get the first 200 000 row of the dataset and save it

In [3]:
if not os.path.exists(path_dataframe + f'ncbi-virus-{n}-dna.*'):
    # Save parquet file in local
    print('Saving subset of dataset...')
    df.to_json(path_dataframe+f'ncbi-virus-{n}-dna.jsonl', index=False, lines=True, orient='records')

Saving subset of dataset...


## For the Multiclass classification task we analize the distribution of the feature target

In [4]:
df['Family'].value_counts()

Family
Geminiviridae       15183
Hepadnaviridae      11360
Orthomyxoviridae    11344
Circoviridae        10076
nan                  9633
                    ...  
Bidnaviridae            1
Dicistroviridae         1
Iflaviridae             1
Qinviridae              1
Steitzviridae           1
Name: count, Length: 101, dtype: int64

Let’s delete the rows that contain nan to the family

In [5]:
index = df[df['Family'] == 'nan'].index
df.drop(index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_079174.1,2023-05-17T00:00:00Z,Erwinia phage Fifi44,Loessnervirus,Chaseviridae,dsDNA,53559,RefSeq,Erwinia amylovora,2020-09,TGTTCCATGTGAGTTACTGAGTTCCGTAATCCTATTATAAAGGCCC...
1,OQ599920.1,2023-06-20T00:00:00Z,Banfec anellovirus 1,Wawtorquevirus,Anelloviridae,ssDNA(-),2471,GenBank,Canis latrans,2021-12-17,CCCCCCCCCTTAAACCCCCCCCGCAGGGGGGCTCCGCCCCCCCTGC...
2,OQ599921.1,2023-06-20T00:00:00Z,Banfec anellovirus 2,Wawtorquevirus,Anelloviridae,ssDNA(-),2479,GenBank,Canis latrans,2021-12-17,CCCCCCCCGCACCCCCCCAAGGGGCTCCGCCCCTTAAACCCCATTG...
3,OQ599922.1,2023-06-20T00:00:00Z,Banfec circovirus 1,Circovirus,Circoviridae,ssDNA(+/-),1680,GenBank,Canis latrans,2021-04-24,TATTACCGGAGACTTTGGAACCGTGGAACAGGCCGATCTGTAATGA...
4,OQ599923.1,2023-06-20T00:00:00Z,Banfec circovirus 1,Circovirus,Circoviridae,ssDNA(+/-),1680,GenBank,Canis latrans,2021-04-23,TATTACCGGAGACTTTGGAACCGTGGAACAGGCCGATCTGTAATGA...


In [6]:
df['Family'].value_counts()

Family
Geminiviridae       15183
Hepadnaviridae      11360
Orthomyxoviridae    11344
Circoviridae        10076
Avsunviroidae        7785
                    ...  
Bidnaviridae            1
Dicistroviridae         1
Iflaviridae             1
Qinviridae              1
Steitzviridae           1
Name: count, Length: 100, dtype: int64

### We select the 4 most frequent classes
We select Geminiviridae, Spinareoviridae, Phenuiviridae and Circoviridae, and save the dataset both to make an unbalanced classification and to oversample the dataset.

In [7]:
selected_df = df.query(
    'Family == "Geminiviridae" or '
    'Family == "Hepadnaviridae" or '
    'Family == "Orthomyxoviridae" or '
    'Family == "Circoviridae"'
)

In [8]:
selected_df['Family'].value_counts()

Family
Geminiviridae       15183
Hepadnaviridae      11360
Orthomyxoviridae    11344
Circoviridae        10076
Name: count, dtype: int64

We check if the dataset has already been saved and save it

In [9]:
if not os.path.exists(path_multiclass+'/UnBalancedDataset.jsonl'):
    print('Saving the dataset...')
    selected_df.to_json(path_multiclass + '/UnBalancedDataset.jsonl', lines=True, orient='records')
    print('Done!')
else:
    print('Dataset already exists')

Dataset already exists


We perform the oversampling with the SMOTEN technique.

The SMOTEN technique is an oversampling technique of minority classes that also works with categorical variables, the imblearn library containing its implementation is used, the reference paper talks about the SMOTE technique in general.

The reference link is the above: [SMOTEN](https://doi.org/10.48550/arXiv.1106.1813) 

In [16]:
cols = selected_df.columns.values.tolist()
cols.remove('Family')
y = selected_df['Family']
X = selected_df[cols]
resampler = SMOTEN(random_state=42)

X,y = resampler.fit_resample(X,y)

In [18]:
X['Family'] = y

X['Family'].value_counts()

Family
Circoviridae        15183
Geminiviridae       15183
Hepadnaviridae      15183
Orthomyxoviridae    15183
Name: count, dtype: int64

In [19]:
X.tail()

,Accession,Release Date,Species,Genus,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence,Family
60727,KF142478.1,2013-06-08T00:00:00Z,Alphainfluenzavirus influenzae,Alphainfluenzavirus,ssRNA(-),2341,GenBank,Anatidae,2008-01,AGCAAAAGCAGGGGAATTTCACAACCAGTCAAAATGGAAACAATATCACTAATAGCTATACTACTAGTAGTAACAGTAAGCAATGCAGATAAAATCTGCATCGGCTACCAATCAACAAACTCCACAGAAACTGTGGATACGCTAACAGAAAACAATGTCCCTGTGACACATGCTAAAGAATTGCTCCACACAGAGCACAATGGGATGCTGTGTGCAACAAATCTGGGACATCCTCTCATTCTAAACACCTGTACCATTGAAGGACTGATCTATGGCAACCCTTCTTGTGATCAGCTGTTGGGAGGAGGAAAATGGTCCTACATCGTCGAAAGACCATCGGCCGTTAATGGAATGTGTTACCCCGGGAATGTAGAAAACCTAGAGGAACTAAGATCACTCTTTAGTTCTGCTAGTTCCTACCAAAAAATTCAGATCTTTCCAGACACGATCTGGAATGTGTCTTACAGTGGAACAAGCAAAGCATGTTCAGATTCATTCTACAGGAGCATGAGATGGCTGACTCAAAAGAACAACGCTTACCCTATTCAAGACGCCCAATACACAAATAATAGAGGAAAGAGCATTCTTTTCATGTGGGGCATAAATCACCCACCCACCGATACTGCACAGACAAATTTGTACACAAGGACCGACACAACAACAAGTGTGACAACAGAAGATATAAATAGAACCTTCAAACCAATGATAGGGCCAAGGCCCCTTGTCAATGGTCAGCAGGGGAGAATTGATTATTATTGGTCGGTATTAAAACCAGGTCAGACATTGCGAATAAGATCCAATGGGAATCTAATTGCTCCATGGTATGGACACATTCTTTCAGGAGAGAGCCACGGAAGAATCCTGAAGACTGATTTAAACAGTGGTAACTGTGTAGTGCAATGTCAGACTGAAAGAGGTGGCTTAAACACCACATTGCCGTTCCACAATGTCAGTAAATATGCATTTGGAAACTGCCCAAAGTATGTTGGAGTAAAGAGTCTCAAACTGGCAGTTGGTCTAAGAAATGTGCCTGCTAGATCAAGTAGAGGACTATTTGGGGCTATAGCTGGTTTCATAGAGGGAGGTTGGTCAGGGTTAGTCGCTGGTTGGTATGGGTTCCAGCATTCAAATGATCAAGGGGTAGGTATGGCTGCAGATAGAGAATCAACTCAAAGGGCAATTGACAAAATAACATCCAAAGTGAATAATATAGTCGATAAAATGAACAAGCAGTATGAAATTATTGATCATGAATTCAGCGAGGTTGAAACTAGACTCAATATGATCAATAATAAGATTGATGATCAAATACAAGACATATGGGCTTATAACGCAGAATTGCTAGTGCTGCTTGAAAATCAGAAAACACTCGATGAACATGATGCAAATGTAAACAATCTATATAACAAAGTGAAGAGGGCACTGGGTTCCAATGCCATGGAAGACGGGAAAGGATGTTTTGAGCTATACCATAAATGTGATGATCAGTGCATGGAGACAATTCGGAACGGGACCTATAACAGGAGAAAGTATAAAGAGGAAGCAAAACTAGAAAGACAGAAAATAGAAGGGGTCAAGCTGGAATCTGAAGGAACTTACAAAATCCTCACCATTTATTCGACTGTCGCCTCATCTCTTGTGATTGCAATGGGGTTTGCTGCCTTCTTGTTCTGGGCCATGTCCAATGGATCTTGCAGATGCAACATTTGTATATAATTGGCAAAAACACCCTTGTTTCTACT,Orthomyxoviridae
60728,JN588806.1,2015-06-30T00:00:00Z,Betainfluenzavirus influenzae,Betainfluenzavirus,ssRNA(-),1024,GenBank,Homo sapiens,2015-02-05,AGCAGAAGCAGAGCATCTTCTCAAAACTGAGGCAAATAGGCCAAAAATGAACAATGCTACCTTCAACTATACAAATGTTAACCCTATTTCTCACATCAGGGGGAGTACTATTATCACTATATGTGTCAGCTTCATTATCATACTTACTATATTCGGATATATTGCTAAAATTCTCACCAACAGAAATAACTGCACCAACAATGCCATTGGATTGTGCAAACGCATCAAATGTTCATGCTGTGAACCGTTCTGCAACAAAAGGGGTGACATTTCTTCTCCCAGAACCGGAGTGGACATACCCGCGTTTATCTTGCCCGGGCTCAACCTTTCAGAAAGCACTCCTAATTAGCCCTCATAGATTCGGAGAAACCAAAGGAAACTCAGCTCCCTTGATAATAAGGGAACCTTTTATTGCTTGTGGACCAAATGAATGCAAACACTTTGCTCTAACCCATTATGCAGCCCAACCAGGGGGATACTACAATGGAACAAGAGGAGACAGAAACAAGCTGAGGCATCTAATTTCAGTCAAATTGGGCAAAATCCCAACAGTAGAAAACTCCATTTTCCACATGGCAGCATGGAGCGGGTCCGCATGCCATGATGGTAAGGAATGGACATATATCGGAGTTGATGGTCCTGACAATAATGCATTGCTCAAAATAAAATATGGAGAAGCATATACTGACACATATCATTCCTATGCAAACAACATCCTAAGAACACAAGAAAGTGCCTGCAATTGCATCGGGGGAAATTGTTATCTTATGATAACTGATGGCTCAGCTTCAGGTGTTAGTGAATGCAGATTTCTTAAGATTCGAGAGGGCCGAATAATAAAAGAAATATTTCCAACAGGAAGAGTAAAACATACTGAGGAGTGCACATGCGGATTTGCCAGCAATAAAACCATAGAATGTGCCTGTAGAGATAACAGTTACACAGCAAAAAGACCCTTTGTCAAATTAAACGTGGAAACTGATACAGCAGAAATAAGATTGATGTGCACAGATACTTATTTGGACACCCCCAGACCAGATGATGGAAGCATAACAGGCCCTTGTGAATCTAATGGGGACAAAGGGAGTGGAGGCATCAAGGGAGGATTTGTTCATCAAAGAATGGCATCCAAGATTGGGAGGTGGTACTCTCGAACGATGTCTAAAACTGAAAGAAAGGGGATGGGACTGTATGTCAAGTATGATGGAGACCCATGGACTGACAGTGATGCCCTTGCTTTTAGGGGAGTAATGGTTTCAATGAAAGAACCTGGTTGGTACTCCTTTGGCTTCGAAATAAAAGATAAGAAATGCGATGTCCCCTGTATTGGGATAGAGATGGTACATGATGGTGGAAAAGAGACTTGGCACTCAGCAGCAACAGCCATTTACTGTTTAATGGGCTCAGGACAGCTGCTGTGGGACACAGTCACAGGTGTCGACATGGCTCTATAATGGAGGAATGGTTGAGTCTGTTCTAAACCCTTTGTTCCTATTTTGTTTGAACAATTGTCCTTACTACACTTAATTGTTTCTGAAAAATGCTCTTGTTACTACAGGTCATAGTGGGTTTCCTGA,Orthomyxoviridae
60729,DQ997515.1,2006-10-09T00:00:00Z,Alphainfluenzavirus influenzae,Alphainfluenzavirus,ssRNA(-),1565,GenBank,nan,nan,AGCAAAAGCAGGAGTTCAAAATGAATCCAAATCAGAAGATAATAACCATCGGATCAATCTGTATGGTAATTGGAATAGTTAGCTTAATGTTACAAATTGGGAACATAATCTCAATATGGGTCAGTCATTCAATTCAGACAGGGAATCAACACCAA

In [21]:
if not os.path.exists(path_multiclass+'/BalancedDataset.jsonl'):
    print('Saving the dataset...')
    X.to_json(path_multiclass + '/BalancedDataset.jsonl', lines=True, orient='records')
    print('Done!')
else:
    print('Dataset already exists')

Saving the dataset...
Done!
